In [3]:
import openmm
sys.path.append("../../")
from enhanced_sampling import utils, system_building as sb, system_saving as ss

In [4]:
input_dict = sb.load_input_dir("../systems/system00", "../forcefield_data/charmm-params/")

Loading psf from ../systems/system00/step5_input.psf
Loading Charmm params from ../forcefield_data/charmm-params/
Loading positions from ../systems/system00/final_frame.cif
Loading box vectors from ../systems/system00/sysinfo.dat
Setting box size x: 121.162246 A, y: 121.162246 A, z:110.56 A


In [5]:
psf = input_dict["psf"]

In [6]:
psf.topology

<Topology; 5 chains, 31576 residues, 151920 atoms, 121416 bonds>

In [7]:
cif = input_dict['cif']

In [8]:
cif.topology

<Topology; 5 chains, 31576 residues, 151920 atoms, 121396 bonds>

In [9]:
params = input_dict['params']

In [10]:
params

In [11]:
param_dict_with_units = sb.load_simulation_params("../sim_params/local_test.yaml")

In [12]:
param_dict_with_units

{'hydrogen_mass': Quantity(value=4.0, unit=dalton),
 'temperature': Quantity(value=303.15, unit=kelvin),
 'friction': Quantity(value=1, unit=/picosecond),
 'time_step': Quantity(value=0.002, unit=picosecond),
 'pressure': Quantity(value=1, unit=bar),
 'surface_tension': 0,
 'nonbonded_method': PME,
 'constraints': HBonds,
 'nsteps': 500000,
 'report_freq': 50000,
 'chk_freq': 250000,
 'traj_freq': 500000,
 'virtual_bond': ['resSeq 46 and name CA and chainid 0',
  'resSeq 46 and name CA and chainid 1'],
 'platform': 'CPU',
 'precision': 'mixed'}

In [14]:
type(param_dict_with_units['nonbonded_method'])

openmm.app.forcefield.PME

In [16]:
type(param_dict_with_units['constraints'])

openmm.app.forcefield.HBonds

In [17]:
system = psf.createSystem(input_dict["params"],
                              nonbondedMethod=param_dict_with_units['nonbonded_method'],
                              constraints=param_dict_with_units['constraints'],
                              removeCMMotion=False,
                              hydrogenMass=param_dict_with_units['hydrogen_mass'])

In [20]:
integrator = openmm.LangevinMiddleIntegrator(param_dict_with_units['temperature'],
                                             param_dict_with_units['friction'],
                                             param_dict_with_units['time_step'])

barostat = openmm.MonteCarloMembraneBarostat(param_dict_with_units['pressure'],
                                         param_dict_with_units['surface_tension'],
                                         param_dict_with_units['temperature'],
                                         openmm.MonteCarloMembraneBarostat.XYIsotropic,
                                         openmm.MonteCarloMembraneBarostat.ZFree
                                         )

In [21]:
barostat.setFrequency(50)  ## for some reason the __init__ won't accept it as an argument, but this works
    ## the default is 25 timesteps, i've set it for 50
system.addForce(barostat)

8

In [22]:
platform = sb.get_platform_from_params(param_dict_with_units)

In [25]:
platform.getName()

'CPU'

In [26]:
sim = openmm.app.Simulation(psf.topology,
                                system=system,
                                integrator=integrator,
                                platform=platform)

In [27]:
sim.context.setPositions(cif.positions)
sim.context.setVelocitiesToTemperature(param_dict_with_units['temperature'])